In [1]:
import sys
import nltk
import numpy as np
import pandas as pd
import pickle
# from helpers import *
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn import metrics
import os
from matplotlib import pyplot as plt
sys.path.append(".")
sys.path.append("..")

In [12]:
col_names = ['title','body','category','business_service','urgency','impact']

dfTickets = pd.read_csv(
    './datasets/all_tickets.csv',
    dtype=str,
    names=col_names
)  

dfTickets.head

<bound method NDFrame.head of                                                                                             title  \
title                                                                                        body   
service unavailable                             sent monday accessible hi we unable access thanks   
for us not working                              hello advise there some issues with open for u...   
not responding                                  sent wednesday down hello down we cannot work ...   
performance issues and                          sent thursday july re incident warning for hi ...   
...                                                                                           ...   
windows upgrade data backup                     re upgrade has started hi increase drive make ...   
windows running slow                            running slow dear help clean install os yester...   
access for environments                         environments 

In [5]:
print(dfTickets.shape)

(43108, 6)


In [9]:
column_to_predict = "impact"
text_columns = "category"

labelData = dfTickets[[column_to_predict]]
data = dfTickets[[text_columns]]

In [10]:
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labelData, test_size=0.2
) 

count_vect = CountVectorizer()

vectorized_data = count_vect.fit_transform(train_data)
vectorized_data.shape

In [11]:
text_clf = Pipeline([
#     ('vect', count_vect),
#     ('tfidf', TfidfTransformer()),
    ('samp', RandomOverSampler()),
    ('clf', MultinomialNB(fit_prior=True))
])
text_clf = text_clf.fit(train_data, train_labels)

AttributeError: 'DataFrame' object has no attribute 'name'

In [ ]:
# parameters = {
#         'vect__ngram_range': [(1, 1), (1, 2)],
#         'tfidf__use_idf': (True, False)
#     }

# # Next, we create an instance of the grid search by passing the classifier, parameters
# # and n_jobs=-1 which tells to use multiple cores from user machine.


# gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)


# gs_clf = gs_clf.fit(train_data, train_labels)

# # To see the best mean score and the params, run the following code
# gs_clf.best_score_
# gs_clf.best_params_

In [ ]:
print("Evaluating model")
# Score and evaluate model on test data using model without hyperparameter tuning
predicted = text_clf.predict(test_data)
prediction_acc = np.mean(predicted == test_labels)
print("Confusion matrix without GridSearch:")
print(metrics.confusion_matrix(test_labels, predicted))
print("Mean without GridSearch: " + str(prediction_acc))


# # Score and evaluate model on test data using model WITH hyperparameter tuning
# predicted = gs_clf.predict(test_data)
# prediction_acc = np.mean(predicted == test_labels)
# print("Confusion matrix with GridSearch:")
# print(metrics.confusion_matrix(test_labels, predicted))
# print("Mean with GridSearch: " + str(prediction_acc))

In [ ]:
# Ploting confusion matrix with 'seaborn' module
# Use below line only with Jupyter Notebook
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
mat = confusion_matrix(test_labels, predicted)
plt.figure(figsize=(4, 4))
sns.set()
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(test_labels),
            yticklabels=np.unique(test_labels))
plt.xlabel('true label')
plt.ylabel('predicted label')
# Save confusion matrix to outputs in Workbench
# plt.savefig(os.path.join('.', 'outputs', 'confusion_matrix.png'))
plt.show()

In [ ]:
%matplotlib inline
from sklearn.metrics import classification_report
print(classification_report(test_labels, predicted,
                            target_names=np.unique(test_labels)))